***GENERATED CODE FOR regressionpipelinehousingsales PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Bldg_Type', 'transformation_label': 'String Indexer'}], 'feature': 'Bldg_Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'Duplex', 'max': 'TwoFmCon', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Bldg_Type'}, {'feature_label': 'Bldg_Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Bldg_Type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'House_Style', 'transformation_label': 'String Indexer'}], 'feature': 'House_Style', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'One_Story', 'max': 'Two_and_Half_Unf', 'missing': '0', 'distinct': '7'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'House_Style'}, {'feature_label': 'House_Style', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('House_Style')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Overall_Cond', 'transformation_label': 'String Indexer'}], 'feature': 'Overall_Cond', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Above_Average', 'max': 'Very_Good', 'missing': '0', 'distinct': '8'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Overall_Cond'}, {'feature_label': 'Overall_Cond', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Overall_Cond')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Exter_Cond', 'transformation_label': 'String Indexer'}], 'feature': 'Exter_Cond', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Excellent', 'max': 'Typical', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Exter_Cond'}, {'feature_label': 'Exter_Cond', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Exter_Cond')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run regressionpipelinehousingsalesHooks.ipynb
try:
	#sourcePreExecutionHook()

	housing = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/housing.csv', 'filename': 'housing.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(housing)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run regressionpipelinehousingsalesHooks.ipynb
try:
	#transformationPreExecutionHook()

	regressionpipelinehousingsalesautofe = TransformationMain.run(housing,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "Lot_Frontage", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "56.54", "stddev": "34.15", "min": "0", "max": "174", "missing": "0"}, "updatedLabel": "Lot_Frontage"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Lot_Area", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "9620.66", "stddev": "5276.88", "min": "1476", "max": "57200", "missing": "0"}, "updatedLabel": "Lot_Area"}, {"transformationsData": [{"feature_label": "Bldg_Type", "transformation_label": "String Indexer"}], "feature": "Bldg_Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Duplex", "max": "TwoFmCon", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Bldg_Type"}, {"transformationsData": [{"feature_label": "House_Style", "transformation_label": "String Indexer"}], "feature": "House_Style", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "One_Story", "max": "Two_and_Half_Unf", "missing": "0", "distinct": "7"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "House_Style"}, {"transformationsData": [{"feature_label": "Overall_Cond", "transformation_label": "String Indexer"}], "feature": "Overall_Cond", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Above_Average", "max": "Very_Good", "missing": "0", "distinct": "8"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Overall_Cond"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Year_Built", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1979.49", "stddev": "26.55", "min": "1875", "max": "2010", "missing": "0"}, "updatedLabel": "Year_Built"}, {"transformationsData": [{"feature_label": "Exter_Cond", "transformation_label": "String Indexer"}], "feature": "Exter_Cond", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Excellent", "max": "Typical", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Exter_Cond"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Total_Bsmt_SF", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1095.92", "stddev": "439.8", "min": "0", "max": "3206", "missing": "0"}, "updatedLabel": "Total_Bsmt_SF"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "First_Flr_SF", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1164.83", "stddev": "399.21", "min": "372", "max": "2696", "missing": "0"}, "updatedLabel": "First_Flr_SF"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Second_Flr_SF", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "359.03", "stddev": "431.84", "min": "0", "max": "1788", "missing": "0"}, "updatedLabel": "Second_Flr_SF"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Full_Bath", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.66", "stddev": "0.53", "min": "0", "max": "4", "missing": "0"}, "updatedLabel": "Full_Bath"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Half_Bath", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.45", "stddev": "0.51", "min": "0", "max": "2", "missing": "0"}, "updatedLabel": "Half_Bath"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Bedroom_AbvGr", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "2.85", "stddev": "0.76", "min": "0", "max": "6", "missing": "0"}, "updatedLabel": "Bedroom_AbvGr"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Kitchen_AbvGr", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "1.04", "stddev": "0.19", "min": "1", "max": "2", "missing": "0"}, "updatedLabel": "Kitchen_AbvGr"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Fireplaces", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.66", "stddev": "0.64", "min": "0", "max": "3", "missing": "0"}, "updatedLabel": "Fireplaces"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Longitude", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "-93.64", "stddev": "0.02", "min": "-93.692521", "max": "-93.599791", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Longitude"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Latitude", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "42.04", "stddev": "0.02", "min": "41.986647", "max": "42.063304", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Latitude"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Sale_Price", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "191387.63", "stddev": "78451.21", "min": "55000", "max": "610000", "missing": "0"}, "updatedLabel": "Sale_Price"}]}))

	#transformationPostExecutionHook(regressionpipelinehousingsalesautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run regressionpipelinehousingsalesHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(regressionpipelinehousingsalesautofe, ["Lot_Frontage", "Lot_Area", "Year_Built", "Total_Bsmt_SF", "First_Flr_SF", "Second_Flr_SF", "Full_Bath", "Half_Bath", "Bedroom_AbvGr", "Kitchen_AbvGr", "Fireplaces", "Longitude", "Latitude", "Bldg_Type_stringindexer", "House_Style_stringindexer", "Overall_Cond_stringindexer", "Exter_Cond_stringindexer"], "Sale_Price")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

